In [ ]:
# -*- coding: utf-8 -*-
"""
Generate audio augmentations based on the folder Musicians Exports

Created on Sun Feb 21 15:09:36 2021
@author: Eli7 Guahnich
"""
Debug_mode = False
GOOG = True
if GOOG:
    !pip install audiomentations
    !pip install soundfile
    from google.colab import drive
    drive.mount('/content/drive')
    root_path= '/content/drive/My Drive/data/MusiciansExports/' #Google_Colab
    generic_path = "/content/drive/My Drive/data/MusiciansExports/I0003/*/*.aif" 
    out_path= '/content/drive/My Drive/data/Augmented/' #Google_Colab
else:
    root_path= "..\\data\\Musicians Exports\\"
    generic_path = "..\\data\\Musicians Exports\\*\\*\\*.aif" 
    out_path= '\\content\\drive\\My Drive\\data\\Augmented\\'

from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import numpy as np
import librosa
import librosa.display as librosa_display
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.io import wavfile
#import sounddevice as sd
import os
from tqdm import tqdm
import glob
import aifc
import pandas as pd



RATES = [0.7, 0.8, 0.9, 0.95, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.5, 1.55, 1.6, 1.7 ]# 0.11, 0.2, 0.3, 0.4, 0.5,
SEMITONES =[-1, -4, 2, 4 ]
SAMPLE_RATE = 16000*60
cols=['filename','nchannels', 'sampwidth', 'framerate', 'nframes', 'comptype', 'compname','markers']
outputfile= root_path + 'augmented_aifflist.csv'

num_rates = len(RATES)
num_semitones = len(SEMITONES)
total_each_project = (num_rates*num_semitones) *3 + num_rates #Drumps just use TimeStretching

print(f'Generate {num_rates} rates and {num_semitones} semitones aprox {total_each_project} new files per project')

def extract_markers(filename):
    """ Extract markers from a file
        :param filename: string, name of the original file
        return a List with the original markers
    """
    if Debug_mode:
        print('Extract markers: ',filename)
    this = None
    try:
        aiff= aifc.open(filename)
        aiffparams=aiff.getparams()
        aiffmarkers=aiff.getmarkers()
        this=[[filename,aiff.getnchannels(),aiff.getsampwidth(),aiff.getframerate(),aiff.getnframes(),
              aiff.getcomptype(),aiff.getcompname(),aiff.getmarkers()]]
        aiff.close()
    except:
        print ("Error extracing markers...", filename) 
    return this

def calculate_save_markers(this_list, current_rate, filename):
    """ Calculate new markers from a list
        :param this_list: list, list of originals markers
        :param current_rate: int rate used for Time Stretching
        return a List with the NEW markers
    """
    new_list =[]
    this_list[0][0] = filename
    if this_list[0][7] != None:
        for each in this_list[0][7]:
            new_position = int(each[1] * current_rate)
            new_subset_marker = (each[0], new_position, each[2])
            new_list.append(new_subset_marker)
        this_list[0].pop(7)
        this_list[0].append(new_list)
    return this_list    
    
    

def generate_augmented_file (augment, data, subfolder, str_, file_info):
     """ Generate and save on disk the new augmented sound file
        :param this_list: augment, object Compose from the library audioaugmentation
        :param data: numpy array,  Original file sound converted to numpy
        :param subfolder: string,  Name of the folder after the root_path
        :param str_: string,  Name of the new folder for the new augmented data
        :param file_info: list,  List with all the original sound file descriptors
        return a string with the augmented new file name 
    """
     output_path = out_path  + subfolder + '/' + str_
     os.makedirs(output_path, exist_ok=True)
     new_file_name = output_path + '/'+file_name
     aa = glob.glob(new_file_name)
     if len(aa) == 0:
         augmented_samples = augment(samples=data, sample_rate=SAMPLE_RATE)
         sf.write(new_file_name, augmented_samples, samplerate= file_info.samplerate, subtype=file_info.subtype, endian=file_info.endian, format=file_info.format,closefd=True)
     #sd.play(augmented_samples, file_info.samplerate, blocking=True)
     if Debug_mode:
         print ("Augmented file:",  output_path + '/'+file_name)
     return new_file_name
    
def load_files (data_path,file_name, subfolder,project_name): 
    """ For a required set of semitones and rates generate the new data augmentation
        :param data_path: original sound file's path
        :param file_name: original sound file's name
        :param subfolder: string,  Name of the folder after the root_path
        :param project_name: string,  Name of next folder after the subfolder
        return a pandas dataframe  with the new markers of the generated sound files.
    """
    aiffs = pd.DataFrame()
    the_markers = extract_markers(data_path + file_name)
    data, sr = sf.read(data_path + file_name)
    file_info = sf.info(data_path + file_name, verbose=False)
    #str_markers = file_info.extra_info
    if Debug_mode:
        if file_name =="I0001_T1_P0002.aif" or file_name =="I0001_T2_P0002.aif":
            return
    for each_rate in RATES:
        #Drumps
        if file_name.find("_T4_") != -1 :
            augment = Compose([
            TimeStretch(min_rate=each_rate, max_rate=each_rate, p=1) #TimeStretch(min_rate=0.8, max_rate=1.25, p=1.0), #Shift(min_fraction=-0.5, max_fraction=0.5, p=1.0),
            ])
            str_ = project_name + 'T0' + str(int(each_rate*100)) 
            new_file_name =generate_augmented_file (augment,data, subfolder, str_, file_info)
            #Generate markers in the csv file.
            markers = calculate_save_markers(the_markers, each_rate, new_file_name)
            aiffs = aiffs.append(markers, ignore_index=True)
        else:       
            for each_semitone in SEMITONES:
                augment = Compose([
                TimeStretch(min_rate=each_rate, max_rate=each_rate, p=1), #TimeStretch(min_rate=0.8, max_rate=1.25, p=1.0), #Shift(min_fraction=-0.5, max_fraction=0.5, p=1.0),
                PitchShift(min_semitones=each_semitone, max_semitones = each_semitone, p=0.5)
                ])
                str_ =  project_name + 'T0' + str(int(each_rate*100))+ 'P0' + str(int(each_semitone*10))
                new_file_name = generate_augmented_file (augment,data, subfolder, str_, file_info)
                #Generate markers in the csv file.
                markers = calculate_save_markers(the_markers, each_rate, new_file_name)
                aiffs = aiffs.append(markers, ignore_index=True)
    if Debug_mode:         
        print ("Complete")
    return aiffs
        
index=1
df_list = []
for path in tqdm(glob.glob(generic_path)):
  if Debug_mode:
    print("Reading...:", path)
  index = index+ 1
  if index<= 0:
    continue
  file_name = path.split('/')[-1]
  sub_folder = path.split('/')[-3] 
  project_name = path.split('/')[-2] 
  data_path = path.replace(file_name,"")
  aiffs= load_files (data_path,file_name, sub_folder, project_name)
  df_list.append(aiffs)
  if Debug_mode:
      if index==4:
           break
makers_list= pd.concat(df_list)
makers_list.columns=cols
makers_list.to_csv(outputfile)


    





     
    

Mounted at /content/drive
Generate 14 rates and 4 semitones aprox 182 new files per project


100%|██████████| 280/280 [52:06<00:00, 11.16s/it]
